# The Cardinal Sin of ML, and Pipes

## After this lecture, you

- **Know that you shouldn't apply preprocessing transformations with info from the testing dataset, that's called "data leakage" and is akin to letting your model "seeing the future" while training**
- Know that you should apply the **exact** transformations to the testing data that you applied to the training data before making predictions

All of these can be accomplished by using `pipelines`. **Pipelines are a crucial ingredient for any viable real-world ML project.**

##  The Cardinal Sin of ML: Data leakage

Now you know how to transform your data before training a model. You might be tempted to do something like:

```python
import #a bunch of sklearn stuff
X, y = #load data
X = transform(X) # imputation, encode cat vars, standardize

# and then you either do these lines:
Xtrain, Xtest, ytrain, ytest = train_test_split(X, y, random_state=9,train_size=.8)
model = # something
model.fit(Xtrain, ytrain)
y_predict = model.predict(Xtest) # using X2 (out-of-sample data), predict y2
accuracy_score(ytest, y_predict)

# or this:
cross_validate(model,X,y)
```

The problem here is that `transform(X)` used info from the **ENTIRE** dataset, including observations that ended up in `Xtest`!

**This means that your cross-validation scores are unreliable.** They will be at the very least overoptimistic, and in some cases, result in models that are down-right completely invalid. 

---

### An illustrative aside

Here is a tiny example of that code in action. Suppose that the dataset has three observations, where X is time, and y is a stock price:

<br> <center> <b> RAW DATA</b> </center>

| X | y | sample |
| :--- | :--- | :--- |
| 1| 1| training
| nan | 2| training |
| 3 | 3 | test | 

Suppose `transform(X)` computes the mean of X and fills in missing values with that. So it figures out that the mean of X is 2 and fills it in. (Remember: Your code above calls `transform` on all of the data, including the test subsample!) So you have, after running `transform(X)`:

<br> <center> <b> AFTER <code>transform(X)</code>, using the code above </b> </center>

|X | y | sample |
| :--- | :--- | :--- |
| 1| 1| training
| **2** | 2| training |
| 3 | 3 | test | 

Now, you split the data up into training and test datasets, and in the training data, you estimate that $y=x$ (a 45 degree line). Thus, you load the test data, see that $X=3$, and predict $Y=3$ and viola! A perfect model!

**HOWEVER.** The test sample is supposed to be data you do not have access to while training the model. E.g. in a  real-world project trying to predict stock prices, $X=3$ occurs next month, so you could have never filled in the value with 2, because you never see $X=3$ while training. Instead, you probably would have filled in $X=1$, the average in the training set, as a best guess:

<br> <center> <b> REAL WORLD DATA AFTER <code>transform(X)</code> WITHOUT SEEING THE FUTURE </b> </center>

| X | y | sample |
| :--- | :--- | :--- |
| 1| 1| training
| **1** | 2| training |

So, this data would led you to conclude that $y=1.5*x$. Thus, when next month arrives, $X=3$, you predict $y=4.5$, and your prediction model is much less accurate than the code above would suggest.

---


### The absolute golden rule of prediction modeling is...

**YOUR MODEL CAN'T HAVE ACCESS TO ANY DATA THAT IT WOULDN'T HAVE IN PRACTICE WHEN IT MAKES THE PREDICTION.**

I know I already said that, and repetition is usually bad writing, but it must be said again. And again.

### Data leakage can be tricky

Here are some more examples of data leakage:
- The outcome variable is a predictor (implicitly or explicitly)
- Predictor variables that are in response to the result (after the fact) or the possibility (anticipatory)
- Predicting loan default, the data might include employee IDs for recent customer service contacts. But the most recent contact might be with trouble-loan specialists (because the firm anticipated possible default due to some other signal). Using that employee's customer contacts to predict default would add no value - the lender already knew to assign that employee!
- The smell test - is it too good to be true? I've seen some asset pricing models with suspicious out-of-sample R2s. Predicting stock prices is hard! _The best OOS predictive R2 for individual stocks [in this paper](https://dachxiu.chicagobooth.edu/download/ML.pdf) is 1.80% per month._

## The solution, or: Safety first, via Pipelines

**Avoiding Data Leakage:**
1. Be very familiar with the data and how it was collected and built 
1. Do your data prep within CV folds

\#2 is [_relatively_ easy to implement in `sklearn`: USE PIPES](https://scikit-learn.org/stable/modules/compose.html)!
- Pipelines make apply all steps to the data they receive
- In `cross_validate`'s training fold, the entire pipeline is applied to the training data
- In `cross_validate`'s testing fold, the saved transformations and model fits are applied to the test data
- We will talk about pipelines for the next two lectures, so set expectations for yourself, work through all the examples, and try to follow the conceptual steps.

Today, let's quickly get our first pipe set up by following this walkthrough on [scaling the iris data and building a classification model](https://chrisalbon.com/machine_learning/model_evaluation/cross_validation_pipeline/)

In [6]:
from sklearn.datasets import load_iris
from sklearn.pipeline import make_pipeline
from sklearn import preprocessing
from sklearn.model_selection import cross_validate
from sklearn import svm

iris = load_iris() # data

# set up the pipeline, which will, given a set of observations 
# 1. fit and apply these steps to the training fold
# 2. in the testing fold, apply the transform and model to predict (no estimation)

classifier_pipeline = make_pipeline(preprocessing.StandardScaler(), svm.SVC(C=1))

# ok, go!
scores = cross_validate(classifier_pipeline, iris.data, iris.target, cv=5)
scores


{'fit_time': array([0.00099659, 0.00099683, 0.00099683, 0.00099778, 0.00199175]),
 'score_time': array([0.        , 0.        , 0.        , 0.00099707, 0.        ]),
 'test_score': array([0.96666667, 0.96666667, 0.96666667, 0.93333333, 1.        ])}

### Some resources on pipes - Examples and walkthroughs:
- [PDSH has an example of imputing, creating polynomial features, then fitting a regression in one line](https://jakevdp.github.io/PythonDataScienceHandbook/05.04-feature-engineering.html)
    - [Another walkthrough with a pipeline](https://jakevdp.github.io/PythonDataScienceHandbook/05.06-linear-regression.html)
    - [A pipeline used to optimize model parameters](https://jakevdp.github.io/PythonDataScienceHandbook/05.07-support-vector-machines.html)
- [sklearn doc with details](https://scikit-learn.org/stable/modules/compose.html)
- [sklearn doc with two walkthroughs](https://scikit-learn.org/stable/tutorial/statistical_inference/putting_together.html), these walkthroughs use a pipeline to optimize a model's parameters


## NEXT YEAR

- copy the exercises from the L22 livecoding in
- remove this stuff form the prior lecture on preprocessing. really improving the other preprocessing stuff (illustrative examples on the value of preprocessing - e.g. better R2, avoid the tradegy of not making predictions because missing values weren't included...
- next lecture - pipeline optimization, diagnostics, and multiple variable types (that's a whole lecture)